In [ ]:
from Bio import SeqIO
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import roc_curve, confusion_matrix, precision_score, accuracy_score, f1_score, matthews_corrcoef, recall_score,auc,precision_recall_curve
import os
feature_name_list = ['18pp','AAC','BPBaac','CTDC','CTDT','CTriad','onehot',
                'PC-PseAAC','ppt25','QSO','SC-PseAAC','CTDD','DPC']
data = {'feature': '','batch':'','bac_name':'','stand':'', 'model': '', 'rate':'','rocAUC': '', 'prAUC': '', 'MCC': '', 'F1': '', 
        'Precision': '', 'Accuracy': '', 'Sensitivity': '', 'Specificity': '', 'FPR': '', 'Recall': '','pro_cutoff':''}
df = pd.DataFrame(columns=data.keys())

rate_list = ['1_1','1_2','1_5','1_10','1_15','1_20','1_22','1_25','1_30']

bac_name = ['Ralstonia_pseudosolanacearum_GMI1000','Salmonella_LT2'][0]
stand = ['lossen','strict'][0]



fasta_file =f"/mnt/md0/Public/T3_T4/blast_out/{bac_name}/{stand}/{bac_name}_blast.fasta"
protein_ids = []
for seq_record in SeqIO.parse(fasta_file, "fasta"):
    protein_id = seq_record.id
    protein_ids.append(protein_id)
for feature_name in feature_name_list:
    batch = 0
    while batch <5:
        for model_name in ["XGBClassifier", "GaussianNB", "GradientBoostingClassifier",   
                                "SVC", "GaussianProcessClassifier", "KNeighborsClassifier", 
                                "RandomForestClassifier"]:
            for rate in rate_list:
                model_save_dir = f"/mnt/md0/Public/T3_T4/model/last_model/{feature_name}/{rate}/{batch}"
                
                if feature_name=='BPBaac':
                    val_df = pd.read_csv(f'/mnt/md0/Public/T3_T4/txseml_addon/out/libfeatureselection/val_data/BPBaac/{batch}/new_{bac_name}_{rate}.fasta_{feature_name}.csv')
                else:
                    val_df = pd.read_csv(f'/mnt/md0/Public/T3_T4/txseml_addon/out/libfeatureselection/val_data/new_{bac_name}.fasta_{feature_name}.csv')
                val_df1 = val_df.iloc[0:, 1:]
                target_list = val_df['protein_id']
                target = []
                for a in range(len(target_list)):
                    if target_list[a] in protein_ids:
                        target.append(1)
                    else:
                        target.append(0)
                feature = pd.DataFrame(val_df1)
                if feature_name == 'CTriad':
                    feature_ = np.array([eval(row) for row in feature['CTriad']])
                else:
                    feature_ = feature.astype("float").values
                    target_ = np.reshape(target, (len(target), 1))
                predict_result_list = []

                
                model = pickle.load(open(f"{model_save_dir}/{model_name}.pkl", "br"))

                pred = model.predict_proba(feature_)[:, 1]
                
                def calculate_fpr(y_true, y_pred):
                    tn, fp, fn, tp = confusion_matrix(y_true,y_pred).ravel()
                    fpr = fp / (fp + tn)
                    return fpr
                fpr, tpr, thresholds = roc_curve(target_, pred)
                
                
                best_one_optimal_idx = np.argmax(tpr - fpr)
                pro_cutoff = thresholds[best_one_optimal_idx]
                
                pred_l = [1 if i >= pro_cutoff else 0 for i in pred]
                #后面新增的计算prAUC
                confusion_matrix_1d = confusion_matrix(target_, pred_l).ravel()
                confusion_dict = {N: n for N, n in zip(['tn', 'fp', 'fn', 'tp'], list(
                    confusion_matrix_1d * 2 / np.sum(confusion_matrix_1d)))}
                
                FPR = calculate_fpr(target_,pred_l)
                
                precision, recall, _ = precision_recall_curve(target_, pred)
                pr_auc = auc(recall, precision)
                Recall = recall_score(target_, pred_l)
                evaluation = {
                    'feature': feature_name, 
                    'batch':batch,
                    'bac_name':bac_name,
                    'stand':stand,
                    'model': model_name,
                    'rate':f'{rate}',
                    "rocAUC": auc(fpr, tpr),
                    "prAUC": pr_auc,
                    "MCC": matthews_corrcoef(target_, pred_l),
                    "F1": f1_score(target_, pred_l),
                    "Precision": precision_score(target_, pred_l,zero_division=1),
                    "Accuracy": accuracy_score(target_, pred_l),
                    "Sensitivity": confusion_dict['tp'] / (confusion_dict['tp'] + confusion_dict['fn']),
                    "Specificity": confusion_dict['tn'] / (confusion_dict['tn'] + confusion_dict['fp']),
                    "FPR":FPR,
                    "Recall":Recall,
                    'pro_cutoff': pro_cutoff
                }

                df = pd.concat([df, pd.DataFrame(evaluation, index=[0])], ignore_index=True)
        batch+=1
# folder_path = f'/mnt/md0/Public/T3_T4/txseml_addon/out/评估/all'
# if not os.path.exists(folder_path):
#     os.makedirs(folder_path)

df.to_excel(f'{bac_name}_{stand}.xlsx', index=False)

5FV

In [ ]:
from Bio import SeqIO
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import roc_curve, confusion_matrix, precision_score, accuracy_score, f1_score, matthews_corrcoef, recall_score,auc,precision_recall_curve
import os
feature_name_list = ['18pp','AAC','BPBaac','CTDC','CTDT','CTriad','DPC','onehot','CTDD',
                'PC-PseAAC','ppt25','QSO','SC-PseAAC']
data = {'feature': '','batch':'', 'model': '', 'rate':'','rocAUC': '', 'prAUC': '', 'MCC': '', 'F1': '', 
        'Precision': '', 'Accuracy': '', 'Sensitivity': '', 'Specificity': '','pro_cutoff':''}
df = pd.DataFrame(columns=data.keys())
prot_type = ['T3','T4','T1','T2','T6'][4]
cd_hit = [30,50,70][1]
rate_list = ['1_1','1_10','1_30','1_50','1_80','1_100']
for feature_name in feature_name_list:
    batch = 0
    while batch <5:
        for rate in rate_list:
            model_save_dir = f"/mnt/md0/Public/T3_T4/model/{prot_type}/{cd_hit}_model/{feature_name}/{rate}/{batch}"
            result = pd.read_csv(f'{model_save_dir}/5fold_results.csv')
            for a in range(len(result)):
                if result.iloc[a,0] == 'GaussianProcessClassifier':
                    continue
                else:
                    evaluation = {
                        'feature': feature_name, 
                        'batch':batch,
                        'model': result.iloc[a,0],
                        'rate':f'{rate}',
                        "rocAUC": result.iloc[a,5],
                        "prAUC": result.iloc[a,9],
                        "MCC": result.iloc[a,4],
                        "F1": result.iloc[a,3],
                        "Precision": result.iloc[a,2],
                        "Accuracy": result.iloc[a,1],
                        "Sensitivity": result.iloc[a,7],
                        "Specificity": result.iloc[a,6],
                        'pro_cutoff': result.iloc[a,8]
                    }
                    df = pd.concat([df, pd.DataFrame(evaluation, index=[0])], ignore_index=True)
        batch+=1


df.to_excel(f'/mnt/md0/Public/T3_T4/评估/{prot_type}/{prot_type}_{cd_hit}_5FV.xlsx', index=False)

bac_5FV

In [11]:
from Bio import SeqIO
import pandas as pd
import numpy as np



feature_name_list = ['18pp','AAC','BPBaac','CTDC','CTDT','CTriad','onehot',
                'PC-PseAAC','ppt25','QSO','SC-PseAAC','CTDD','DPC']
data = {'feature': '','bac_name':'', 'rate':'','model':'','rocAUC': '', 'prAUC': '', 'MCC': '', 'F1': '', 
        'Precision': '', 'Accuracy': '', 'Sensitivity': '', 'Specificity': '', 'pro_cutoff':''}
df = pd.DataFrame(columns=data.keys())
bac_name_list=['30_Acinetobacter_baumannii_K09_14.fasta','30_Advenella_kashmirensis_WT001.fasta','30_Agrobacterium_tumefaciens_12D1.fasta',
               '30_Escherichia_coli_MG1655.fasta','30_Klebsiella_pneumoniae_subsp.pneumoniae_HS11286.fasta','30_Stenotrophomonas_sp.CD2.fasta']
rate_dict = {
    '30_Acinetobacter_baumannii_K09_14.fasta':13.1,
    '30_Advenella_kashmirensis_WT001.fasta':12.7,
    '30_Agrobacterium_tumefaciens_12D1.fasta':16.6,
    '30_Escherichia_coli_MG1655.fasta':15.1,
    '30_Klebsiella_pneumoniae_subsp.pneumoniae_HS11286.fasta':18.4,
    '30_Stenotrophomonas_sp.CD2.fasta':13.1
}
rate_list = ['1_1','1_2','1_5','1_10','1_15','1_20','1_22','1_25','1_30']
for feature_name in feature_name_list:
    for model_bac_name in bac_name_list:
        model_save_dir = f"/mnt/md0/Public/T3_T4/model/bac_last_model/{model_bac_name}/{feature_name}"
        result = pd.read_csv(f'{model_save_dir}/5fold_results.csv')
        for a in range(len(result)):
            evaluation = {
                'feature': feature_name, 
                'bac_name':model_bac_name,
                
                'rate':rate_dict[f'{model_bac_name}'],
                'model': result.iloc[a,0],
                "rocAUC": result.iloc[a,5],
                "prAUC": result.iloc[a,9],
                "MCC": result.iloc[a,4],
                "F1": result.iloc[a,3],
                "Precision": result.iloc[a,2],
                "Accuracy": result.iloc[a,1],
                "Sensitivity": result.iloc[a,7],
                "Specificity": result.iloc[a,6],
                'pro_cutoff': result.iloc[a,8]
            }
            df = pd.concat([df, pd.DataFrame(evaluation, index=[0])], ignore_index=True)


df.to_excel(f'bac_5FV.xlsx', index=False)

In [7]:
import pandas as pd
import os
from scipy import stats
import matplotlib.pyplot as plt

import numpy as np
model = ["XGBClassifier", "GaussianNB", "GradientBoostingClassifier",   
                                "SVC", "KNeighborsClassifier", 
                                "RandomForestClassifier"][4]
all = pd.read_excel(f'/mnt/md0/Public/T3_T4/评估/T3/30_duli.xlsx')
fv = pd.read_excel(f'/mnt/md0/Public/T3_T4/评估/T3/30_5FV.xlsx')
# all = pd.read_excel('/mnt/md0/Public/T3_T4/独立验证.xlsx')
# fv = pd.read_excel('/mnt/md0/Public/T3_T4/5FV.xlsx')
rate = list(all['rate'])
fv_rate = list(fv['rate'])
rocAUC = list(all['rocAUC'])
prAUC = list(all['prAUC'])
MCC = list(all['MCC'])
F1 = list(all['F1'])

Precision = list(all['Precision'])
Accuracy = list(all['Accuracy'])
Sensitivity = list(all['Sensitivity'])
Specificity = list(all['Specificity'])

fv_rocAUC = list(fv['rocAUC'])
fv_prAUC = list(fv['prAUC'])
fv_MCC = list(fv['MCC'])
fv_F1 = list(fv['F1'])

fv_Precision = list(fv['Precision'])
fv_Accuracy = list(fv['Accuracy'])
fv_Sensitivity = list(fv['Sensitivity'])
fv_Specificity = list(fv['Specificity'])

lists = [
prAUC, 
]

fv_lists = [
fv_prAUC, 
]

name_lists = ['rocAUC',
'prAUC', 
'MCC' ,
'F1' ,
'Precision',
'Sensitivity' ,
'Specificity']


for a,lst in enumerate(lists):
    one_1 = []
    one_2 = []
    one_5 = []
    one_10 = []
    one_20 = []
    one_30 = []
    one_40 = []
    one_50 = []
    one_60 = []
    one_70 = []
    one_80 = []
    one_100 = []
    
    fv_one_1 = []
    fv_one_2 = []
    fv_one_5 = []
    fv_one_10 = []
    fv_one_20 = []
    fv_one_30 = []
    fv_one_40 = []
    fv_one_50 = []
    fv_one_60 = []
    fv_one_70 = []
    fv_one_80 = []
    fv_one_100 = []
    for b in range(len(fv)):
        if fv_rate[b] == '1_1':
            fv_one_1.append(fv_lists[a][b])
        elif fv_rate[b] == '1_10':
            fv_one_10.append(fv_lists[a][b])
        elif fv_rate[b] == '1_30':
            fv_one_30.append(fv_lists[a][b])
        elif fv_rate[b] == '1_50':
            fv_one_50.append(fv_lists[a][b])
        elif fv_rate[b] == '1_80':
            fv_one_80.append(fv_lists[a][b])
        elif fv_rate[b] == '1_100':
            fv_one_100.append(fv_lists[a][b])
        
    fv_one_1 = np.mean(fv_one_1)
    fv_one_10 = np.mean(fv_one_10)
    fv_one_30 = np.mean(fv_one_30)
    fv_one_50 = np.mean(fv_one_50)
    fv_one_80 = np.mean(fv_one_80)
    fv_one_100 = np.mean(fv_one_100)
    print(fv_one_1,fv_one_10,fv_one_30,fv_one_50,fv_one_80,fv_one_100)
    b = 0
    for b in range(len(all)):
        if rate[b] == '1_1':
            one_1.append(lst[b])
        elif rate[b] == '1_10':
            one_10.append(lst[b])
        elif rate[b] == '1_30':
            one_30.append(lst[b])
        elif rate[b] == '1_50':
            one_50.append(lst[b]) 
        elif rate[b] == '1_80':
            one_80.append(lst[b]) 
        elif rate[b] == '1_100':
            one_100.append(lst[b])
        


0.856428811812949 0.5093316698571038 0.30502749940668716 0.2236689991911914 0.16791566402277633 0.14319498502561667
